# Import

In [240]:
library(GNAR)

In [897]:
library(jsonlite)

# function

In [1318]:
cal_mse <- function(data, net_input, lags, missing_rate) {
      train_ratio <- floor(nrow(data) * 0.8)
      test_ratio <- ceiling(nrow(data) * 0.2)
      df <- data
      num_cols <- ncol(df)
      for (i in 1:num_cols) {
              indices_to_replace <- sample(c(1:(nrow(df) - test_ratio - lags)), 
                                           size = missing_rate * (nrow(df) - test_ratio - lags))
              df[indices_to_replace, i] <- NA
            }
      nafit <- GNARfit(vts = df[1:train_ratio,], net = net_input, alphaOrder = lags, betaOrder = rep(1, lags))
      predict<- predict(nafit,n.ahead = test_ratio)
    
      mse_each <- numeric(num_cols)
      for (i in 1:num_cols) {
          mse_each[i] <- mean((predict[,i] - df[(train_ratio+1):nrow(df),i])**2)
      }
      mse_total <- mean(mse_each)
      return(list('each'= mse_each, 'total'=mse_total))
    }

In [1344]:
block_cal_mse <- function(data, net_input, lags, mindex) {
      train_ratio <- floor(nrow(data) * 0.8)
      test_ratio <- ceiling(nrow(data) * 0.2)
      df <- data
      num_cols <- ncol(df)
      for (i in 1:num_cols) {
              df[mindex[[i]],i] <- NA
            }
      nafit <- GNARfit(vts = df[1:train_ratio,], net = net_input, alphaOrder = lags, betaOrder = rep(1, lags))
      predict<- predict(nafit,n.ahead = test_ratio)
    
      mse_each <- numeric(num_cols)
      for (i in 1:num_cols) {
          mse_each[i] <- mean((predict[,i] - df[(train_ratio+1):nrow(df),i])**2)
        }
      mse_total <- mean(mse_each)
      return(list('each'= mse_each, 'total'=mse_total))
    }

# fivenodes

## random

In [1330]:
fixed_data <- fiveVTS
fixed_net_input <- fiveNet
fixed_lags <- 2

results_df <- data.frame(mrate = numeric(),
                         mse = numeric())

missing_rates <- c(0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8) 

for (rate in missing_rates) {
  result <- cal_mse(fixed_data, fixed_net_input, fixed_lags, rate)
  results_df <- rbind(results_df, data.frame(mrate = rate, mse = result[2]))
}
results_df

mrate,total
<dbl>,<dbl>
0.0,1.257729
0.1,1.252735
0.2,1.265524
0.3,1.250937
0.4,1.271402
0.5,1.278707
0.6,1.301504
0.7,1.301504
0.8,1.301504


In [1331]:
results_df['dataset'] = 'FiveVTS'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'rand'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
FiveVTS_results <- results_df

## block

In [1345]:
FiveVTS_mindex <- fromJSON("FiveVTS_mindex.json")

In [1348]:
fixed_data <- fiveVTS
fixed_net_input <- fiveNet
fixed_lags <- 2
fixed_mindex <- FiveVTS_mindex

results_df <- data.frame(mrate = numeric(),
                         mse = numeric())

total_length <- 0

for (i in seq_along(fixed_mindex)) {
  total_length <- total_length + length(fixed_mindex[[i]])
}

rate = total_length/(length(fixed_data[,])*0.8)

results_df <- data.frame(mrate = rate,
                         mse = block_cal_mse(fixed_data, fixed_net_input, fixed_lags, fixed_mindex)[2])
results_df

mrate,total
<dbl>,<dbl>
0.125,1.272369


In [1349]:
results_df['dataset'] = 'FiveVTS'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'block'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
FiveVTS_block_results <- results_df

# Chickenpox

## random

In [1351]:
Chickenpox<-read.csv('./data/Chickenpox.csv',header = TRUE)

In [1352]:
Chickenpox_w<-read.csv('./data/Chickenpox_w.csv',header = FALSE)[2:21,2:21]

In [1385]:
fixed_data <- as.matrix(Chickenpox[,2:ncol(Chickenpox)])
fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(Chickenpox_w))
fixed_lags <- 4

results_df <- data.frame(mrate = numeric(),
                         mse = numeric())

missing_rates <- c(0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8)

for (rate in missing_rates) {
  result <- cal_mse(fixed_data, fixed_net_input, fixed_lags, rate)
  results_df <- rbind(results_df, data.frame(mrate = rate, mse = result[2]))
}
results_df

mrate,total
<dbl>,<dbl>
0.0,1.016342
0.1,1.016460
0.2,1.015845
0.3,1.018481
0.4,1.020843
0.5,1.019158
0.6,1.021600
0.7,1.021411
0.8,1.020084


In [1386]:
results_df['dataset'] = 'Chickenpox'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'rand'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
Chickenpox_results <- results_df

## block

In [1387]:
Chickenpox_mindex <- fromJSON("Chickenpox_mindex.json")

In [1388]:
fixed_data <- as.matrix(Chickenpox[,2:ncol(Chickenpox)])
fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(Chickenpox_w))
fixed_lags <- 4
fixed_mindex <- Chickenpox_mindex

total_length <- 0

for (i in seq_along(fixed_mindex)) {
  total_length <- total_length + length(fixed_mindex[[i]])
}

rate = total_length/(length(fixed_data[,])*0.8)

results_df <- data.frame(mrate = rate,
                         mse = block_cal_mse(fixed_data, fixed_net_input, fixed_lags, fixed_mindex)[2])
results_df

mrate,total
<dbl>,<dbl>
0.2884615,1.017119


In [1389]:
results_df['dataset'] = 'Chickenpox'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'block'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
Chickenpox_block_results <- results_df

# PedalMe

## random

In [1390]:
PedalMe<-read.csv('./data/PedalMe.csv',header = TRUE)

In [1391]:
PedalMe_w<-read.csv('./data/PedalMe_w.csv',header = FALSE)[2:16,2:16]

In [1455]:
fixed_data <- as.matrix(PedalMe[,2:ncol(PedalMe)])
fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(PedalMe_w))
fixed_lags <- 4

results_df <- data.frame(mrate = numeric(),
                         mse = numeric())

missing_rates <- c(0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6,0.7, 0.8) 

for (rate in missing_rates) {
  result <- cal_mse(fixed_data, fixed_net_input, fixed_lags, rate)
  results_df <- rbind(results_df, data.frame(mrate = rate, mse = result[2]))
}
results_df

mrate,total
<dbl>,<dbl>
0.0,0.5016028
0.1,0.4971018
0.2,0.5027820
0.3,0.5148579
0.4,0.5226205
0.5,0.5917800
0.6,0.4917371
0.7,0.4917371
0.8,0.4917371


In [1456]:
results_df['dataset'] = 'Pedalme'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'rand'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
Pedalme_results <- results_df

## block

In [1457]:
Pedalme_mindex <- fromJSON("Pedalme_mindex.json")

In [1458]:
fixed_data <- as.matrix(PedalMe[,2:ncol(PedalMe)])
fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(PedalMe_w))
fixed_lags <- 4
fixed_mindex <- Pedalme_mindex

total_length <- 0

for (i in seq_along(fixed_mindex)) {
  total_length <- total_length + length(fixed_mindex[[i]])
}

rate = total_length/(length(fixed_data[,])*0.8)

results_df <- data.frame(mrate = rate,
                         mse = block_cal_mse(fixed_data, fixed_net_input, fixed_lags, fixed_mindex)[2])
results_df

mrate,total
<dbl>,<dbl>
0.2941176,0.4916068


In [1459]:
results_df['dataset'] = 'Pedalme'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'block'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
Pedalme_block_results <- results_df

# WikiMath

## random

In [1460]:
WikiMath <- read.csv('./data/WikiMath.csv',header = TRUE)

In [1461]:
WikiMath_w<-read.csv('./data/WikiMath_w.csv',header = FALSE)[2:1069,2:1069]

In [ ]:
fixed_data <- as.matrix(WikiMath[,2:ncol(WikiMath)])
fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(WikiMath_w))
fixed_lags <- 8

results_df <- data.frame(mrate = numeric(),
                         mse = numeric())

missing_rates <- c(0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,0.8) 

for (rate in missing_rates) {
  result <- cal_mse(fixed_data, fixed_net_input, fixed_lags, rate)
  results_df <- rbind(results_df, data.frame(mrate = rate, mse = result[2]))
}
results_df

In [ ]:
results_df['dataset'] = 'Wikimath'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'rand'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
Wikimath_results <- results_df

## block

In [1465]:
Wikimath_mindex <- fromJSON("Wikimath_mindex.json")

In [1466]:
fixed_data <- as.matrix(WikiMath[,2:ncol(WikiMath)])
fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(WikiMath_w))
fixed_lags <- 8
fixed_mindex <- Wikimath_mindex

total_length <- 0

for (i in seq_along(fixed_mindex)) {
  total_length <- total_length + length(fixed_mindex[[i]])
}

rate = total_length/(length(fixed_data[,])*0.8)

results_df <- data.frame(mrate = rate,
                         mse = block_cal_mse(fixed_data, fixed_net_input, fixed_lags, fixed_mindex)[2])
results_df

mrate,total
<dbl>,<dbl>
0.1202478,0.9009986


In [1467]:
results_df['dataset'] = 'Wikimath'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'block'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
Wikimath_block_results <- results_df

# Windmillsmall

## random

In [1468]:
Windmillsmall<-read.csv('./data/Windmillsmall.csv',header = TRUE)

In [1469]:
Windmillsmall_w<-read.csv('./data/Windmillsmall_w.csv',header = FALSE)[2:12,2:12]

계속 오류남;;

In [885]:
# fixed_data <- as.matrix(Windmillsmall[,2:ncol(Windmillsmall)])
# fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(Windmillsmall_w))
# fixed_lags <- 8

# results_df <- data.frame(mrate = numeric(),
#                          mse = numeric())

# missing_rates <- c(0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,0.8) 

# for (rate in missing_rates) {
#   result <- cal_mse(fixed_data, fixed_net_input, fixed_lags, rate)
#   results_df <- rbind(results_df, data.frame(mrate = rate, mse = result))
# }
# results_df

In [1543]:
results_df <- data.frame(
  mrate = c(0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1, 0.0),
  mse = c(0.980465007853614,0.980465007853614,0.980742615896619,0.980465007853614, 0.980529157673243, 0.980548927794843,0.980486712335988,0.980471934178215, 0.980486581012802)
)

In [1544]:
results_df['dataset'] = 'Windmillsmall'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'rand'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
Windmillsmall_results <- results_df

In [1505]:
fixed_data <- as.matrix(Windmillsmall[,2:ncol(Windmillsmall)])
fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(Windmillsmall_w))
fixed_lags <- 8

In [1524]:
cal_mse(fixed_data, fixed_net_input, fixed_lags, 0.8)[2]

$total
[1] 0.980465

In [1541]:
cal_mse(fixed_data, fixed_net_input, fixed_lags, 0.7)[2]

$total
[1] 0.980465

In [1527]:
cal_mse(fixed_data, fixed_net_input, fixed_lags, 0.6)[2]

$total
[1] 0.9807426

In [1512]:
cal_mse(fixed_data, fixed_net_input, fixed_lags, 0.5)[2]

$total
[1] 0.980465

In [1511]:
cal_mse(fixed_data, fixed_net_input, fixed_lags, 0.4)[2]

$total
[1] 0.9805292

In [1510]:
cal_mse(fixed_data, fixed_net_input, fixed_lags, 0.3)[2]

$total
[1] 0.9805489

In [1509]:
cal_mse(fixed_data, fixed_net_input, fixed_lags, 0.2)[2]

$total
[1] 0.9804867

In [1508]:
cal_mse(fixed_data, fixed_net_input, fixed_lags, 0.1)[2]

$total
[1] 0.9804719

In [1507]:
cal_mse(fixed_data, fixed_net_input, fixed_lags, 0)[2]

$total
[1] 0.9804866

## block

In [1221]:
Windmillsmall_mindex <- fromJSON("Windmillsmall_mindex.json")

In [1503]:
fixed_data <- as.matrix(Windmillsmall[,2:ncol(Windmillsmall)])
fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(Windmillsmall_w))
fixed_lags <- 8
fixed_mindex <- Windmillsmall_mindex

total_length <- 0

for (i in seq_along(fixed_mindex)) {
  total_length <- total_length + length(fixed_mindex[[i]])
}

rate = total_length/(length(fixed_data[,])*0.8)

results_df <- data.frame(mrate = rate,
                         mse = block_cal_mse(fixed_data, fixed_net_input, fixed_lags, fixed_mindex)[2])
results_df

mrate,total
<dbl>,<dbl>
0.03252143,0.9804863


In [1504]:
results_df['dataset'] = 'Windmillsmall'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'block'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
Windmillsmall_block_results <- results_df

# MontevideoBus

## random

In [877]:
MontevideoBus<-read.csv('./data/MontevideoBus.csv',header = TRUE)

In [878]:
MontevideoBus_w<-read.csv('./data/MontevideoBus_w.csv',header = FALSE)[2:676,2:676]

In [1498]:
fixed_data <- as.matrix(MontevideoBus[,2:ncol(MontevideoBus)])
fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(MontevideoBus_w))
fixed_lags <- 8

results_df <- data.frame(mrate = numeric(),
                         mse = numeric())

missing_rates <- c(0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7,0.8) 

for (rate in missing_rates) {
  result <- cal_mse(fixed_data, fixed_net_input, fixed_lags, rate)
  results_df <- rbind(results_df, data.frame(mrate = rate, mse = result[2]))
}
results_df

mrate,total
<dbl>,<dbl>
0.0,1.002868
0.1,1.002744
0.2,1.002750
0.3,1.002771
0.4,1.001533
0.5,1.002278
0.6,1.002135
0.7,1.003126
0.8,1.003196


In [1499]:
results_df['dataset'] = 'MontevideoBus'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'rand'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
MontevideoBus_results <- results_df

## block

In [1226]:
MontevideoBus_mindex <- fromJSON("MontevideoBus_mindex.json")

In [1501]:
fixed_data <- as.matrix(MontevideoBus[,2:ncol(MontevideoBus)])
fixed_net_input <- GNAR::matrixtoGNAR(as.matrix(MontevideoBus_w))
fixed_lags <- 8
fixed_mindex <- MontevideoBus_mindex

total_length <- 0

for (i in seq_along(fixed_mindex)) {
  total_length <- total_length + length(fixed_mindex[[i]])
}

rate = total_length/(length(fixed_data[,])*0.8)

results_df <- data.frame(mrate = rate,
                         mse = block_cal_mse(fixed_data, fixed_net_input, fixed_lags, fixed_mindex)[2])
results_df

mrate,total
<dbl>,<dbl>
0.1495439,1.002894


In [1502]:
results_df['dataset'] = 'MontevideoBus'
results_df['method'] = 'GNAR'
results_df['mtype'] = 'block'
results_df['lags'] = fixed_lags
results_df['nof_filters'] = NA
results_df['inter_method'] = NA
results_df['epoch'] = NA
results_df['calculation_time'] = NA
results_df['model'] = 'GNAR'
colnames(results_df)[2] <- "mse"
MontevideoBus_block_results <- results_df

# Final Data

In [1269]:
rbind(FiveVTS_results,Chickenpox_results,Pedalme_results,Wikimath_results,Windmillsmall_results,MontevideoBus_results)

mrate,mse,dataset,method,mtype,lags,nof_filters,inter_method,epoch,calculation_time,model
<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>
0.0,1.234698e+00,FiveVTS,GNAR,rand,2,NA,NA,NA,NA,GNAR
0.1,1.244323e+00,FiveVTS,GNAR,rand,2,NA,NA,NA,NA,GNAR
0.2,1.243565e+00,FiveVTS,GNAR,rand,2,NA,NA,NA,NA,GNAR
0.3,1.259479e+00,FiveVTS,GNAR,rand,2,NA,NA,NA,NA,GNAR
0.4,1.262327e+00,FiveVTS,GNAR,rand,2,NA,NA,NA,NA,GNAR
0.5,1.249555e+00,FiveVTS,GNAR,rand,2,NA,NA,NA,NA,GNAR
0.6,1.245251e+00,FiveVTS,GNAR,rand,2,NA,NA,NA,NA,GNAR
0.7,1.276700e+00,FiveVTS,GNAR,rand,2,NA,NA,NA,NA,GNAR
0.8,1.276700e+00,FiveVTS,GNAR,rand,2,NA,NA,NA,NA,GNAR


In [1270]:
rbind(FiveVTS_block_results,Chickenpox_block_results,Pedalme_block_results,Wikimath_block_results,Windmillsmall_block_results,MontevideoBus_block_results)

mrate,mse,dataset,method,mtype,lags,nof_filters,inter_method,epoch,calculation_time,model
<dbl>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<lgl>,<lgl>,<lgl>,<lgl>,<chr>
0.12500000,1.25466526,FiveVTS,GNAR,block,2,NA,NA,NA,NA,GNAR
0.28846154,1.81270606,Chickenpox,GNAR,block,4,NA,NA,NA,NA,GNAR
0.29411765,0.03161668,Pedalme,GNAR,block,4,NA,NA,NA,NA,GNAR
0.12024781,0.73046199,Wikimath,GNAR,block,8,NA,NA,NA,NA,GNAR
0.03252143,0.98558421,Windmillsmall,GNAR,block,8,NA,NA,NA,NA,GNAR
0.14954389,1.32200400,MontevideoBus,GNAR,block,8,NA,NA,NA,NA,GNAR


In [1271]:
final_df <- rbind(FiveVTS_results,Chickenpox_results,Pedalme_results,Wikimath_results,Windmillsmall_results,MontevideoBus_results,
     FiveVTS_block_results,Chickenpox_block_results,Pedalme_block_results,Wikimath_block_results,Windmillsmall_block_results,MontevideoBus_block_results)

In [1272]:
write.csv(final_df, file = "./R_GNAR_results.csv", row.names = FALSE)